In [26]:
import numpy as np

def compute_K_vectors(as_param, aq_param, cs_param, cq_param, rho_s, rho_q, sigma_squared, eta):
    """
    Compute the K vectors as defined in Equation 22
    
    Parameters:
    -----------
    as_param, aq_param: Parameters for the short-run and long-run volatility components
    cs_param, cq_param: Leverage parameters for the short-run and long-run components
    rho_s, rho_q: Persistence parameters for short-run and long-run components
    sigma_squared: Unconditional variance
    eta: Parameter for IG innovations
    
    Returns:
    --------
    K1, K2, K3, K4, K5: The K vectors as defined in Equation 22
    """

    # Define K2 and K3
    K2 = np.array([as_param, aq_param])
    K3 = np.array([cs_param, cq_param])
    
    # Compute K1
    K1 = np.array([0, sigma_squared * (1 - rho_q)]) - eta**4 * K2
    
    # Compute K4
    K4 = np.array([rho_s, 0]) - eta**2 * K2 - (1/eta**2) * K3
    
    # Compute K5
    K5 = np.array([0, rho_q]) - eta**2 * K2 - (1/eta**2) * K3
    
    return K1, K2, K3, K4, K5

def compute_kappa_values(theta_l, theta_y, K2, K3, eta):
    """
    Compute κ₁ and κ₂ as defined in Equations 25 and 26
    
    Parameters:
    -----------
    theta_l: Vector of factor risk preference parameters [theta_s, theta_q]
    theta_y: Equity risk preference parameter
    K2, K3: K vectors from Equation 22
    eta: Parameter for IG innovations
    lambda_param: Equity risk premium parameter
    
    Returns:
    --------
    kappa1, kappa2: Values of κ₁ and κ₂
    """
    # Calculate kappa1 (Equation 25)
    kappa1 = 1 - 2 * eta**4 * np.dot(theta_l, K2)
    
    # Calculate kappa2 (Equation 26)
    kappa2 = 1 - 2 * theta_y * eta - 2 * np.dot(theta_l, K3)
    
    return kappa1, kappa2

In [29]:
def compute_theta_l(theta_s, theta_q):
    """
    Compute θl based on Equation 21 from the JF paper
    
    Parameters:
    -----------
    theta_s, theta_q : float
        Factor risk preference parameters
    
    Returns:
    --------
    theta_l : array-like
        Vector of factor risk preference parameters [theta_s, theta_q]
    """
    theta_l = np.array([theta_s, theta_q])
    return theta_l

def compute_theta_y(theta_l, lambda_param, eta, K2, K3):
    """
    Compute θy based on Equation 23 from the JF paper
    
    Parameters:
    -----------
    theta_l : array-like
        Vector of factor risk preference parameters [theta_s, theta_q]
    lambda_param : float
        Equity risk premium parameter
    eta : float
        Parameter for IG innovations
    K2, K3 : array-like
        K vectors from Equation 22
    
    Returns:
    --------
    theta_y : float
        Equity risk preference parameter
    """
    # Term 1: (1 - 2θl^T K3)/(2η)
    term1 = (1 - 2 * np.dot(theta_l, K3)) / (2 * eta)
    
    num = lambda_param**2 * eta**3 + 2 * (1 - 2 * eta**4 * np.dot(theta_l, K2))
    denom = 8 * lambda_param**2 * eta**3 * (1 - 2 * eta**4 * np.dot(theta_l, K2))
    term2 = num**2 / denom
    
    # Combine the terms to get θy
    theta_y = term1 - term2
    
    return theta_y

In [19]:
def compute_model_params_and_vectors(eta, lam, K1, K2, K3, K4, K5, kappa1, kappa2):
    """
    Compute the model parameters and vectors for IG_GARCH(C)
    
    Parameters:
    -----------
    eta: Parameter for IG innovations
    lam: Equity risk premium parameter
    K1, K2, K3, K4, K5: K vectors from Equation 22
    kappa1, kappa2: Values of κ₁ and κ₂ from Equations 25 and 26
    
    Returns:
    --------
    eta*, lambda*, K1*, K2*, K3*, K4*, K5*: Model parameters and vectors for IG_GARCH(C)
    """
    
    # Compute eta*
    eta_star = eta / kappa2

    # Compute lambda*
    lam_star = lam * np.sqrt((kappa2**3)/kappa1)

    # Compute K1*
    K1_star = K1 * np.sqrt(kappa1/(kappa2**3))

    # Compute K2*
    K2_star = K2 * np.sqrt((kappa2**5)/kappa1)

    # Compute K3*
    K3_star = K3 * np.sqrt(kappa1/(kappa2**5))

    # Compute K4*
    K4_star = K4

    # Compute K5*
    K5_star = K5
    
    return eta_star, lam_star, K1_star, K2_star, K3_star, K4_star, K5_star

In [100]:
import functools # for memoizing the recursive functions for B_s* and B_q*
def compute_A_and_B_star_functions(**params):
    

    eta_star = params['eta_star']
    lam_star = params['lam_star']
    K1_star = params['K1_star']
    K2_star = params['K2_star']
    K3_star = params['K3_star']
    K4_star = params['K4_star']
    K5_star = params['K5_star']
    r = params["r"]



    def compute_B_star(u, v, t, T):
        """
        Compute the B* function for IG_GARCH(C)
        
        Parameters:
        -----------
        u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
        t, T: Time periods
        
        Returns:
        --------
        B_star: Vector generated from B_s* and B_q* components
        """
        
        if t == T:
            return v
        else:
            return np.array([compute_B_s_star(u, v, t, T), compute_B_q_star(u, v, t, T)])




    def compute_B_s_star(u, v, t, T):
        """
        Compute the B_s* function for IG_GARCH(C)
        
        Parameters:
        -----------
        u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
        t, T: Time periods

        Returns:
        --------
        B_s_star: Value of the B_s* function
        """
        
        if t == T:
            return 0
        else:
            Bstar_tplus1 = compute_B_star(u, v, t+1, T)

            term1 = 1 - 2 * eta_star**4 * np.dot(Bstar_tplus1, K2_star)
            term2 = 1 - 2 * u * eta_star - 2 * np.dot(Bstar_tplus1, K3_star)
            root = np.sqrt(term1 * term2)

            return u*lam_star + np.dot(Bstar_tplus1, K4_star) + (1 - root)/eta_star**2
        

    def compute_B_q_star(u, v, t, T, **params):
        """
        Compute the B_q* function for IG_GARCH(C)
        
        Parameters:
        -----------
        u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
        t, T: Time periods

        Returns:
        --------
        B_q_star: Value of the B_q* function
        """
        
        if t == T:
            return 0
        else:

            Bstar_tplus1 = compute_B_star(u, v, t+1, T)

            term1 = 1 - 2 * eta_star**4 * np.dot(Bstar_tplus1, K2_star)
            term2 = 1 - 2 * u * eta_star - 2 * np.dot(Bstar_tplus1, K3_star)
            root = np.sqrt(term1 * term2)

            return u*lam_star + np.dot(Bstar_tplus1, K5_star) + (1 - root)/eta_star**2
        

    def compute_B_0_star(u, v, t, T):
        return 0


    def compute_A_star(u, v, t, T):
        """
        Compute the A* function for IG_GARCH(C)
        
        Parameters:
        -----------
        u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
        t, T: Time periods

        
        Returns:
        --------
        A_star: Value of A*
        """

        if t == T:
            return 0
        else:
            Astar_tplus1 = compute_A_star(u, v, t+1, T)
            Bstar_tplus1 = compute_B_star(u, v, t+1, T)
            log_term = 1 - 2*eta_star**4 * np.dot(Bstar_tplus1, K2_star)
            return Astar_tplus1 + u*r + np.dot(Bstar_tplus1, K1_star) - 0.5 * np.log(log_term)
    
    return compute_A_star, compute_B_star, compute_B_0_star
            

In [102]:
def psi_0(t, y_t, l_star_t, r, A_star, B0_star, B_star):
    """
    Compute psi_0_t value as defined in Proposition 2
    """
    zero = np.zeros(2)

    # Computing A_star(2, 0; t, t+1)
    A_star_val = A_star(2, zero, t, t+1)
    
    # Computing B0_star(2, 0; t, t+1)
    B0_star_val = B0_star(2, zero, t, t+1)
    
    # Computing B_star(2, 0; t, t+1)
    B_star_val = B_star(2, zero, t, t+1)
    

    dot_product = np.dot(B_star_val, l_star_t)
    
    # Computing psi_0_t
    psi_0_t = np.exp(-2*r + A_star_val + B0_star_val * y_t + dot_product) - 1
    
    return psi_0_t

def psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T):
    """
    Compute psi_1_t(z) value as defined in Proposition 2
    """
    zero = np.zeros(2)

    # Computing A_star(z, 0; t, T)
    A_star_val = A_star(z, zero, t, T)
    
    # Computing B0_star(z, 0; t, T)
    B0_star_val = B0_star(z, zero, t, T)
    
    # Computing B_star(z, 0; t, T)
    B_star_val = B_star(z, zero, t, T)
    

    dot_product = np.dot(B_star_val, l_star_t)
        
    psi_1_val = np.exp(A_star_val + B0_star_val * y_t + dot_product)
    
    return psi_1_val

def psi_2(t, z, y_t, l_star_t, r, A_star, B0_star, B_star, T):
    """
    Compute psi_2_t(z) value as defined in Proposition 2
    """
    zero = np.zeros(2)

    # Computing A_star(z, 0; t+1, T)
    A_star_val_1 = A_star(z, zero, t+1, T)
    
    # Computing B0_star(z, 0; t+1, T)
    B0_star_val = B0_star(z, zero, t+1, T)
    
    # Computing B_star(z, 0; t+1, T)
    B_star_val = B_star(z, zero, t+1, T)
    
    # Computing A_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    A_star_val_2 = A_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Computing B0_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    B0_star_val_2 = B0_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Computing B_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    B_star_val_2 = B_star(1 + z + B0_star_val, B_star_val, t, t+1)
    

    dot_product = np.dot(B_star_val_2, l_star_t)

    
    # Computing psi_2_t(z)
    psi_2_val = np.exp(-r + A_star_val_1 + A_star_val_2) * \
                np.exp(B0_star_val_2 * y_t) * \
                np.exp(dot_product)
    
    return psi_2_val

In [77]:
def integrand(z, t, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, T):
    """
    Compute the integrand for equation (8)
    
    Parameters:
    z: complex variable for the integration
    t: time point
    y_t: log-return at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    
    Returns:
    Integrand value
    """
    # Computing e^((z-1)*Y_t)
    exp_term = np.exp((z-1) * y_t)
    
    # Computing psi_2_t(z) - psi_1_t(z)
    psi1 = psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T)
    psi2 = psi_2(t, z, y_t, l_star_t, r, A_star, B0_star, B_star, T)
    psi_diff = psi2 - psi1

    
    # Computing f_check(z)
    f_check_val = f_check(z, **option_params)

    
    # Computing the integrand
    result = exp_term * psi_diff * f_check_val
    
    return result

In [25]:
def f_check_call(z, K):
    """
    Inverse Laplace transform of a European call option payoff
    
    Parameters:
    z: complex variable for the integration
    K: strike price
    
    Returns:
    Inverse Laplace transform value
    """
    return K**(1-z) / (z * (z - 1))

def f_check_put(z, K):
    """
    Inverse Laplace transform of a European put option payoff
    
    Parameters:
    z: complex variable for the integration
    K: strike price
    
    Returns:
    Inverse Laplace transform value
    """
    return K**(1-z) / (z * (z - 1)) * (-1)  # The put option payoff has opposite sign

In [36]:
def compute_l_t_star(s_t, q_t, delta):
    """
    Compute the scaled factor vector at time t
    
    Parameters:
    s_t: Short-run component at time t
    q_t: Long-run component at time t
    delta: Scaling factor
    
    Returns:
    l_t_star: Scaled factor vector at time t
    """
    l_t_star = np.array([s_t, q_t]) * delta
    return l_t_star

In [51]:
from scipy import integrate

def risk_minimizing_hedge(t, T, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, R):
    """
    Compute the risk-minimizing hedging position using equation (8)
    
    Parameters:
    t: current time point
    T: option maturity
    y_t: log-return at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    R: Real part of the contour in the complex plane
    
    Returns:
    Risk-minimizing hedging position
    """
    # Computing psi_0_t
    psi_0_t = psi_0(t, y_t, l_star_t, r, A_star, B0_star, B_star)
    
    # Defining the complex contour for numerical integration
    # We'll use a contour that goes vertically through R + i*y for y in [-N, N]
    N = 100  # Limit for numerical integration
    
    # Defining the integrand function for the given parameters
    def integrand_for_quad(y):
        z = complex(R, y)
        ret = integrand(z, t, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, T).real
        return ret
    
    # Computing the integral using numerical integration
    integral_result, _ = integrate.quad(integrand_for_quad, -N, N)
    
    # Computing the final result with the complex i in the denominator
    # Since we're taking the real part of the integrand, we need to account for the i in the denominator
    # When dividing by i, it's equivalent to multiplying by -i
    # This rotates the result by -90 degrees in the complex plane
    xi_t_plus_1 = np.exp(-r * (T - t)) / (2 * np.pi * psi_0_t) * (-1) * integral_result
    
    return xi_t_plus_1

In [109]:
def option_price(t, T, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, R):
    """
    Compute the option price using equation (9)
    
    Parameters:
    t: current time point
    T: option maturity
    y_t: log of asset price at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    R: Real part of the contour in the complex plane
    
    Returns:
    Option price
    """
    # Defining the complex contour for numerical integration
    # We'll use a contour that goes vertically through R + i*y for y in [-N, N]
    N = 100  # Limit for numerical integration
    
    # Defining the integrand function for the given parameters
    def price_integrand(y):
        z = complex(R, y)
        # Computing e^{zY_t}
        exp_term = np.exp(z * y_t)
        
        # Computing ψ^{(1)}_t(z)
        psi_1_val = psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T)
        
        # Computing \check{f}(z)
        f_check_val = f_check(z, **option_params)
        
        # Computing the integrand
        result = exp_term * psi_1_val * f_check_val
        
        return result.real
    
    # Computing the integral using numerical integration
    integral_result, _ = integrate.quad(price_integrand, -N, N, limit=1000)
    
    # Computing the final result
    price = np.exp(-r * (T - t)) * integral_result / (2 * np.pi)
    
    return price

# Testing out my code

In [30]:
lam = 2.6591
sigma_squared = 1.1676e-4
rho_s = 7.9656e-1
a_s = 2.1765e7
c_s = 1.7172e-6
rho_q = 9.9124e-1
a_q = 3.6037e7
c_q = 2.4548e-6
eta = -6.0027e-4
theta_s = 2.6168e4
theta_q = 4.9589e4

In [31]:
print("Computing the K vectors...")
K1, K2, K3, K4, K5 = compute_K_vectors(a_s, a_q, c_s, c_q, rho_s, rho_q, sigma_squared, eta)
print("K1:", K1)
print("K2:", K2)
print("K3:", K3)
print("K4:", K4)
print("K5:", K5)

Computing the K vectors...
K1: [-2.82582477e-06 -3.65598999e-06]
K2: [21765000. 36037000.]
K3: [1.7172e-06 2.4548e-06]
K4: [-11.81160334 -19.79775464]
K5: [-12.60816334 -18.80651464]


In [32]:
print("Computing the θl vector...")
theta_l = compute_theta_l(theta_s, theta_q)
print("θl:", theta_l)

Computing the θl vector...
θl: [26168. 49589.]


In [33]:
print("Computing the θy parameter...")
theta_y = compute_theta_y(theta_l, lam, eta, K2, K3)
print("θy:", theta_y)

Computing the θy parameter...
θy: 126873847.95521061


In [34]:
print("Computing the κ₁ and κ₂ values...")
kappa1, kappa2 = compute_kappa_values(theta_l, theta_y, K2, K3, eta)
print("κ₁:", kappa1)
print("κ₂:", kappa2)

Computing the κ₁ and κ₂ values...
κ₁: 0.3880728560493959
κ₂: 152317.79609061492


In [35]:
print("Computing the model parameters and vectors...")
eta_star, lam_star, K1_star, K2_star, K3_star, K4_star, K5_star = compute_model_params_and_vectors(eta, lam, K1, K2, K3, K4, K5, kappa1, kappa2)
print("η*:", eta_star)
print("λ*:", lam_star)
print("K1*:", K1_star)
print("K2*:", K2_star)
print("K3*:", K3_star)
print("K4*:", K4_star)
print("K5*:", K5_star)

Computing the model parameters and vectors...
η*: -3.9409052350186e-09
λ*: 253748806.0214236
K1*: [-2.96125556e-14 -3.83120738e-14]
K2*: [3.16357880e+20 5.23803763e+20]
K3*: [1.18141069e-19 1.68886965e-19]
K4*: [-11.81160334 -19.79775464]
K5*: [-12.60816334 -18.80651464]


In [115]:
params = {
    'eta_star': eta_star,
    'lam_star': lam_star,
    'K1_star': K1_star,
    'K2_star': K2_star,
    'K3_star': K3_star,
    'K4_star': K4_star,
    'K5_star': K5_star,
    'r': 0.02
}

# Create A*, B0*, and B* functions
A_star, B_star, B0_star = compute_A_and_B_star_functions(**params)

# Define option parameters
option_params = {
    'K': 90  # Strike price
}

# Current state
t = 0
T = 1
y_t =  np.log(100)# Log of return at time t
delta = np.sqrt(kappa1 / kappa2**3)
l_star_t = compute_l_t_star(0, sigma_squared, delta) # long and short run conditional variances at time t
r = params['r']  # Risk-free rate
R = 1.5  # Real part of the contour for European call option (R > 1)

# Compute the risk-minimizing hedging position

xi_t_plus_1 = risk_minimizing_hedge(
    t, T, y_t, l_star_t, r,
    A_star, B0_star, B_star,
    f_check_call, option_params, R
)

print(f"Risk-minimizing hedging position: {xi_t_plus_1}")



# Compute the option price
option_price_value = option_price(
    t, T, y_t, l_star_t, r,
    A_star, B0_star, B_star,
    f_check_call, option_params, R
)

print(f"Option price: {option_price_value}")

Risk-minimizing hedging position: -0.23504651528773685
Option price: 11.786436410184704
